# Create virtual machines in Orbstack(MacOS)/ multipass(Ubuntu)

This setup is meant for running Ansible playbooks on MacOS/Ubuntu. 
For Linux support, set up two virtual machines with multipass, boot up the number of machines you wish to provision, [create required users](https://documentation.opencrvs.org/setup/3.-installation/3.3-set-up-a-server-hosted-environment/3.3.1-provision-your-server-nodes-with-ssh-access#create-a-user-named-provision) and create a variation of `local.macos.yml`/`local.linux.yml` inventory file. 

## Prerequisites for MacOs
1. Install [OrbStack](orbstack.dev)
2. Confirm `orb` command is now available on your command line

## Prerequisites for Ubuntu
1. Install [multipass](https://canonical.com/multipass/install)
2. Confirm installation with `multipass --version` command

Either run the commands manually from command like (for live output) or by executing cells here

Identifying the OS you are currently using

In [62]:
import platform

linux = platform.system() == "Linux"

Running the provision script according to OS

In [63]:
import subprocess
if linux:
    subprocess.run(["bash", "./provision-linux.sh"], check=True)
else:
    subprocess.run(["bash", "./provision-mac.sh"], check=True)


Removing existing VM: manager
Creating manager  Configuring manager  Starting manager  Waiting for initialization to complete  Launched: manager
Creating worker  Configuring worker  Starting worker  Waiting for initialization to complete  Launched: worker
Provisioning user on manager...
info: Adding group `provision' (GID 1100) ...
info: Adding user `provision' ...
info: Adding new user `provision' (1100) with group `provision (1100)' ...
info: Creating home directory `/home/provision' ...
info: Copying files from `/etc/skel' ...
info: Adding new user `provision' to supplemental / extra groups `users' ...
info: Adding user `provision' to group `users' ...
provision ALL=(ALL) NOPASSWD:ALL
Provisioning user on worker...
info: Adding group `provision' (GID 1100) ...
info: Adding user `provision' ...
info: Adding new user `provision' (1100) with group `provision (1100)' ...
info: Creating home directory `/home/provision' ...
info: Copying files from `/etc/skel' ...
info: Adding new user `p

Notice that the following task will prompt (in VSCode) for your dockerhub username and password

In [64]:
import os

os.environ["DOCKER_USERNAME"] = input("Enter Dockerhub username: ")
os.environ["DOCKER_PASSWORD"] = input("Enter Dockerhub password: ")

The next command is run with Python instead of %%bash simply because %%bash doesn't support real-time output

In [67]:
inventory = 'local.macos.yml'
if linux:
    inventory = 'local.linux.yml'

cmd = f"""
mongo_user=$(openssl rand -hex 8)
mongo_pass=$(openssl rand -base64 24)
backup_pass=$(openssl rand -base64 16)
es_pass=$(openssl rand -base64 24)

ansible-playbook -i {inventory} ../server-setup/playbook.yml \\
  -e encrypted_disk_size=200g \\
  -e dockerhub_username=$DOCKER_USERNAME \\
  -e dockerhub_password=$DOCKER_PASSWORD \\
  -e mongodb_admin_username=$mongo_user \\
  -e mongodb_admin_password="$mongo_pass" \\
  -e backup_encryption_passphrase="$backup_pass" \\
  -e elasticsearch_superuser_password="$es_pass" \\
  -vvv
"""

In [69]:
import subprocess

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True, env=os.environ)
for line in proc.stdout:
    print(line, end="")
proc.wait()

ansible-playbook [core 2.17.6]
  config file = /etc/ansible/ansible.cfg
  configured module search path = ['/home/dsi/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/lib/python3/dist-packages/ansible
  ansible collection location = /home/dsi/.ansible/collections:/usr/share/ansible/collections
  executable location = /usr/bin/ansible-playbook
  python version = 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0] (/usr/bin/python3)
  jinja version = 3.1.2
  libyaml = True
Using /etc/ansible/ansible.cfg as config file
host_list declined parsing /home/dsi/Documents/opencrvs-countryconfig/infrastructure/local-development/local.linux.yml as it did not pass its verify_file() method
script declined parsing /home/dsi/Documents/opencrvs-countryconfig/infrastructure/local-development/local.linux.yml as it did not pass its verify_file() method
[WARNING]: Invalid characters were found in group names but not replaced, use
-vvvv to see details
Pa

4